In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('https://archive.ics.uci.edu/static/public/911/data.csv')

# Display the first few rows of the dataset
print(df.head())

# Check for basic information about the dataset
print(df.info())

   num_records  recipe_number  recipe_code         recipe_name  \
0            0              1        14299  Creamy White Chili   
1            1              1        14299  Creamy White Chili   
2            2              1        14299  Creamy White Chili   
3            3              1        14299  Creamy White Chili   
4            4              1        14299  Creamy White Chili   

                                        comment_id         user_id  \
0  sp_aUSaElGf_14299_c_2G3aneMRgRMZwXqIHmSdXSG1hEM  u_9iFLIhMa8QaG   
1  sp_aUSaElGf_14299_c_2FsPC83HtzCsQAtOxlbL6RcaPbY  u_Lu6p25tmE77j   
2  sp_aUSaElGf_14299_c_2FPrSGyTv7PQkZq37j92r9mYGkP  u_s0LwgpZ8Jsqq   
3  sp_aUSaElGf_14299_c_2DzdSIgV9qNiuBaLoZ7JQaartoC  u_fqrybAdYjgjG   
4  sp_aUSaElGf_14299_c_2DtZJuRQYeTFwXBoZRfRhBPEXjI  u_XXWKwVhKZD69   

    user_name  user_reputation  created_at  reply_count  thumbs_up  \
0     Jeri326                1  1665619889            0          0   
1     Mark467               50  1665277687

In [13]:
# Separate rows with stars equal to 0
unknown_attitudes = df[df['stars'] == 0]
df = df[df['stars'] != 0]

# Display counts
print("Unknown attitudes count:", len(unknown_attitudes))
print("Known attitudes count:", len(df))

# Remove duplicated rows based on 'text' column, keeping only one
df = df.drop_duplicates(subset=['text'])

# Verify duplicates are removed
print(f"Number of rows after removing duplicates: {df.shape[0]}")
import matplotlib.pyplot as plt

Unknown attitudes count: 1696
Known attitudes count: 16486
Number of rows after removing duplicates: 16097


In [14]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Download required NLTK packages
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package punkt to /Users/allenyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/allenyang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/allenyang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
def preprocess_text(text):
    # Handle non-string inputs
    if not isinstance(text, str):
        return ""
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into a single string
    return ' '.join(tokens)

In [16]:
# Apply preprocessing to the 'text' column
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Display a few examples of cleaned text
print("Original Text:")
print(df['text'].head())
print("\nCleaned Text:")
print(df['cleaned_text'].head())

Original Text:
0    I tweaked it a little, removed onions because ...
1    Bush used to have a white chili bean and it ma...
2    I have a very complicated white chicken chili ...
5    amazing! my boyfriend loved it so much! going ...
6    Wow!!!  This recipe is excellent as written!! ...
Name: text, dtype: object

Cleaned Text:
0    tweaked little removed onion onion hater house...
1    bush used white chili bean made recipe super s...
2    complicated white chicken chili recipe made ye...
5         amazing boyfriend loved much going make week
6    wow recipe excellent written change made used ...
Name: cleaned_text, dtype: object


In [17]:
# Check for missing values
print("Missing values before handling:")
print(df.isnull().sum())

# Drop rows with missing 'cleaned_text' or 'stars'
df = df.dropna(subset=['cleaned_text', 'stars'])

# Verify no missing values
print("Missing values after handling:")
print(df.isnull().sum())

Missing values before handling:
num_records        0
recipe_number      0
recipe_code        0
recipe_name        0
comment_id         0
user_id            0
user_name          0
user_reputation    0
created_at         0
reply_count        0
thumbs_up          0
thumbs_down        0
stars              0
best_score         0
text               1
cleaned_text       0
dtype: int64
Missing values after handling:
num_records        0
recipe_number      0
recipe_code        0
recipe_name        0
comment_id         0
user_id            0
user_name          0
user_reputation    0
created_at         0
reply_count        0
thumbs_up          0
thumbs_down        0
stars              0
best_score         0
text               1
cleaned_text       0
dtype: int64


In [18]:
# Remove rows where 'text' or 'cleaned_text' is missing
df = df.dropna(subset=['text', 'cleaned_text'])

# Verify that there are no missing values now
print("Missing values after cleanup:")
print(df.isnull().sum())

Missing values after cleanup:
num_records        0
recipe_number      0
recipe_code        0
recipe_name        0
comment_id         0
user_id            0
user_name          0
user_reputation    0
created_at         0
reply_count        0
thumbs_up          0
thumbs_down        0
stars              0
best_score         0
text               0
cleaned_text       0
dtype: int64


In [26]:
# Define new sentiment mapping
def map_sentiment(stars):
    if stars == 5:
        return 'very positive'
    elif stars == 4:
        return 'positive'
    elif stars == 3:
        return 'neutral'
    elif stars == 2:
        return 'negative'
    elif stars == 1:
        return 'very negative'
    else:
        return None  # Handle invalid values

# Apply the mapping to the 'stars' column
df['sentiment'] = df['stars'].apply(map_sentiment)

# Verify the mapping
print(df[['stars', 'sentiment']].head())



   stars      sentiment
0      5  very positive
1      5  very positive
2      5  very positive
5      5  very positive
6      5  very positive


In [27]:
from gensim.models import Word2Vec
import numpy as np

# Train Word2Vec model
sentences = df['cleaned_text'].apply(lambda x: x.split())
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Generate sentence embeddings by averaging word vectors
def compute_sentence_embedding(sentence, model, embedding_dim=100):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(embedding_dim)

X_embeddings = np.array([compute_sentence_embedding(text, word2vec_model) for text in df['cleaned_text']])
print("Word2Vec Embeddings Shape:", X_embeddings.shape)


Word2Vec Embeddings Shape: (16096, 100)


In [28]:
from sklearn.preprocessing import LabelEncoder

# Encode the target column
encoder = LabelEncoder()
df['encoded_sentiment'] = encoder.fit_transform(df['sentiment'])
print("Encoded Classes:", encoder.classes_)  # ['very negative', 'negative', 'neutral', 'positive', 'very positive']



Encoded Classes: ['negative' 'neutral' 'positive' 'very negative' 'very positive']


In [29]:
# Prepare embeddings for FFN
X_ffn = X_embeddings  # Shape: (batch_size, embedding_dim)

# Reshape embeddings for RNN/LSTM
X_rnn = X_embeddings.reshape((X_embeddings.shape[0], 1, X_embeddings.shape[1]))  # Shape: (batch_size, 1, embedding_dim)

# Encode target labels
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df['sentiment'])


In [30]:
from sklearn.model_selection import train_test_split

# Train-test split for FFN and RNN/LSTM
X_ffn_train, X_ffn_test, y_train, y_test = train_test_split(X_ffn, y, test_size=0.3, random_state=42, stratify=y)
X_rnn_train, X_rnn_test, _, _ = train_test_split(X_rnn, y, test_size=0.3, random_state=42, stratify=y)

print("FFN Input Shape (Train):", X_ffn_train.shape)
print("RNN Input Shape (Train):", X_rnn_train.shape)

FFN Input Shape (Train): (11267, 100)
RNN Input Shape (Train): (11267, 1, 100)


In [31]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, f1_score
import numpy as np

2024-12-11 17:20:19.120432: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [32]:
# Custom callback to compute and log macro F1 score
class MacroF1Score(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data
        self.best_macro_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        val_predictions = np.argmax(self.model.predict(self.validation_data[0]), axis=1)
        val_true = self.validation_data[1]
        macro_f1 = f1_score(val_true, val_predictions, average="macro")
        logs['val_macro_f1'] = macro_f1

        print(f"Epoch {epoch + 1} - Macro F1 Score: {macro_f1:.4f}")

        # Save the best F1 score
        if macro_f1 > self.best_macro_f1:
            self.best_macro_f1 = macro_f1
            print(f"New Best Macro F1 Score: {self.best_macro_f1:.4f}")

In [33]:
def build_ffn_model():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_ffn_train.shape[1],)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(len(encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy')
    return model


In [34]:
def build_rnn_model():
    model = Sequential([
        SimpleRNN(128, input_shape=(1, X_rnn_train.shape[2]), return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy')
    return model

In [35]:
def build_lstm_model():
    model = Sequential([
        LSTM(128, input_shape=(1, X_rnn_train.shape[2]), return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy')
    return model

In [36]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1),
                                 initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(1,),
                                 initializer='zeros', trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

In [37]:
from tensorflow.keras.layers import Input, Bidirectional

def build_attbirnn_model():
    input_layer = Input(shape=(1, X_rnn_train.shape[2]))
    birnn_layer = Bidirectional(LSTM(128, return_sequences=True))(input_layer)
    attention_layer = Attention()(birnn_layer)
    dense_layer = Dense(64, activation='relu')(attention_layer)
    output_layer = Dense(len(encoder.classes_), activation='softmax')(dense_layer)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy')
    return model

In [38]:
# Early stopping with patience of 10 epochs
early_stopping = EarlyStopping(monitor='val_macro_f1', patience=10, mode='max', restore_best_weights=True)

# Reduce learning rate when macro F1 score plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_macro_f1', factor=0.5, patience=5, mode='max', verbose=1)


In [39]:
def train_ffn_and_evaluate():
    model = build_ffn_model()

    # Prepare macro F1 callback
    f1_callback = MacroF1Score(validation_data=(X_ffn_test, y_test))

    # Train the model
    model.fit(
        X_ffn_train, y_train,
        validation_split=0.1,
        epochs=50,
        batch_size=32,
        callbacks=[f1_callback, early_stopping, reduce_lr],
        verbose=1
    )

    # Predict on the test set
    y_ffn_pred = model.predict(X_ffn_test)
    y_ffn_pred_classes = np.argmax(y_ffn_pred, axis=1)

    # Generate classification report
    print("FFN Classification Report:")
    print(classification_report(y_test, y_ffn_pred_classes, target_names=encoder.classes_))

train_ffn_and_evaluate()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   
Epoch 1 - Macro F1 Score: 0.1832
New Best Macro F1 Score: 0.1832
317/317 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1.4962 - val_loss: 0.5656 - val_macro_f1: 0.1832 - learning_rate: 0.0010
Epoch 2/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step loss:
Epoch 2 - Macro F1 Score: 0.1881
New Best Macro F1 Score: 0.1881
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5960 - val_loss: 0.5446 - val_macro_f1: 0.1881 - learning_rate: 0.0010
Epoch 3/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step loss:
Epoch 3 - Macro F1 Score: 0.2083
New Best Macro F1 Score: 0.2083
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5688 - val_loss: 0.5187 - val_macro_f1: 0.2083 - learning_rate: 0.0010
Epoch 4/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step loss
Epoch 4 - Macro F1 Score: 0.2100
New Best Macro F1 Score: 0.2100
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5544 - val_loss: 0.5090 - val_macro_f1: 0.2100 - learning_rate: 0.0010
Epoch 5/50
151/151

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [40]:
def train_rnn_and_evaluate():
    model = build_rnn_model()

    # Prepare macro F1 callback
    f1_callback = MacroF1Score(validation_data=(X_rnn_test, y_test))

    # Train the model
    model.fit(
        X_rnn_train, y_train,
        validation_split=0.1,
        epochs=50,
        batch_size=32,
        callbacks=[f1_callback, early_stopping, reduce_lr],
        verbose=1
    )

    # Predict on the test set
    y_rnn_pred = model.predict(X_rnn_test)
    y_rnn_pred_classes = np.argmax(y_rnn_pred, axis=1)

    # Generate classification report
    print("RNN Classification Report:")
    print(classification_report(y_test, y_rnn_pred_classes, target_names=encoder.classes_))

train_rnn_and_evaluate()


Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   lo
Epoch 1 - Macro F1 Score: 0.2025
New Best Macro F1 Score: 0.2025
317/317 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.7505 - val_loss: 0.5229 - val_macro_f1: 0.2025 - learning_rate: 0.0010
Epoch 2/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step loss: 
Epoch 2 - Macro F1 Score: 0.1939
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5538 - val_loss: 0.5194 - val_macro_f1: 0.1939 - learning_rate: 0.0010
Epoch 3/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - l
Epoch 3 - Macro F1 Score: 0.1972
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5449 - val_loss: 0.5126 - val_macro_f1: 0.1972 - learning_rate: 0.0010
Epoch 4/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - lo
Epoch 4 - Macro F1 Score: 0.2072
New Best Macro F1 Score: 0.2072
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5292 - val_loss: 0.5106 - val_macro_f1: 0.2072 - learning_rate: 0.0010
Epoch 5/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss
Epoch 5 - Macro F1 Score: 0.1888
317/

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [41]:
def train_lstm_and_evaluate():
    model = build_lstm_model()

    # Prepare macro F1 callback
    f1_callback = MacroF1Score(validation_data=(X_rnn_test, y_test))

    # Train the model
    model.fit(
        X_rnn_train, y_train,
        validation_split=0.1,
        epochs=50,
        batch_size=32,
        callbacks=[f1_callback, early_stopping, reduce_lr],
        verbose=1
    )

    # Predict on the test set
    y_lstm_pred = model.predict(X_rnn_test)
    y_lstm_pred_classes = np.argmax(y_lstm_pred, axis=1)

    # Generate classification report
    print("LSTM Classification Report:")
    print(classification_report(y_test, y_lstm_pred_classes, target_names=encoder.classes_))

train_lstm_and_evaluate()


Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   
Epoch 1 - Macro F1 Score: 0.1850
New Best Macro F1 Score: 0.1850
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.7283 - val_loss: 0.5334 - val_macro_f1: 0.1850 - learning_rate: 0.0010
Epoch 2/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - los
Epoch 2 - Macro F1 Score: 0.1834
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5438 - val_loss: 0.5200 - val_macro_f1: 0.1834 - learning_rate: 0.0010
Epoch 3/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoch 3 - Macro F1 Score: 0.1850
New Best Macro F1 Score: 0.1850
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5238 - val_loss: 0.5320 - val_macro_f1: 0.1850 - learning_rate: 0.0010
Epoch 4/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - los
Epoch 4 - Macro F1 Score: 0.1857
New Best Macro F1 Score: 0.1857
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5430 - val_loss: 0.5135 - val_macro_f1: 0.1857 - learning_rate: 0.0010
Epoch 5/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoc

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [43]:
import tensorflow as tf
def train_attbirnn_and_evaluate():
    model = build_attbirnn_model()

    # Prepare macro F1 callback
    f1_callback = MacroF1Score(validation_data=(X_rnn_test, y_test))

    # Train the model
    model.fit(
        X_rnn_train, y_train,
        validation_split=0.1,
        epochs=50,
        batch_size=32,
        callbacks=[f1_callback, early_stopping, reduce_lr],
        verbose=1
    )

    # Predict on the test set
    y_attbirnn_pred = model.predict(X_rnn_test)
    y_attbirnn_pred_classes = np.argmax(y_attbirnn_pred, axis=1)

    # Generate classification report
    print("AttBiRNN Classification Report:")
    print(classification_report(y_test, y_attbirnn_pred_classes, target_names=encoder.classes_))

train_attbirnn_and_evaluate()

Epoch 1/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Epoch 1 - Macro F1 Score: 0.1950
New Best Macro F1 Score: 0.1950
317/317 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.6337 - val_loss: 0.5268 - val_macro_f1: 0.1950 - learning_rate: 0.0010
Epoch 2/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - los
Epoch 2 - Macro F1 Score: 0.1988
New Best Macro F1 Score: 0.1988
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.5275 - val_loss: 0.5285 - val_macro_f1: 0.1988 - learning_rate: 0.0010
Epoch 3/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoch 3 - Macro F1 Score: 0.2039
New Best Macro F1 Score: 0.2039
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5233 - val_loss: 0.5088 - val_macro_f1: 0.2039 - learning_rate: 0.0010
Epoch 4/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - los
Epoch 4 - Macro F1 Score: 0.2113
New Best Macro F1 Score: 0.2113
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5112 - val_loss: 0.5129 - val_macro_f1: 0.2113 - learning_rate: 0.0010
Epoch 5/50
151/151 ━━━

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [46]:
from sklearn.model_selection import ParameterSampler
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import LeakyReLU
from sklearn.metrics import f1_score
import numpy as np
from scipy.stats import randint, uniform


In [47]:
# Hyperparameter ranges
param_distributions = {
    'neurons': randint(64, 256),  # For FFN and Dense layers in RNN/LSTM
    'units': randint(64, 256),  # For RNN/LSTM/AttBiRNN layers
    'dropout': uniform(0.2, 0.5),
    'learning_rate': uniform(0.0001, 0.01),
    'activation': ['relu', 'tanh', 'sigmoid', 'leaky_relu'],  # Activation functions
    'optimizer': ['adam', 'rmsprop', 'sgd']  # Optimizers
}

In [48]:
def build_ffn_model_tuned(neurons, dropout, learning_rate, activation, optimizer):
    activation_fn = LeakyReLU() if activation == 'leaky_relu' else activation
    opt = {'adam': Adam, 'rmsprop': RMSprop, 'sgd': SGD}[optimizer](learning_rate=learning_rate)

    model = Sequential([
        Dense(neurons, activation=activation_fn, input_shape=(X_ffn_train.shape[1],)),
        Dropout(dropout),
        Dense(neurons // 2, activation=activation_fn),
        Dropout(dropout),
        Dense(len(encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy')
    return model


In [49]:
def build_rnn_model_tuned(units, dropout, learning_rate, activation, optimizer):
    activation_fn = LeakyReLU() if activation == 'leaky_relu' else activation
    opt = {'adam': Adam, 'rmsprop': RMSprop, 'sgd': SGD}[optimizer](learning_rate=learning_rate)

    model = Sequential([
        SimpleRNN(units, activation=activation_fn, input_shape=(1, X_rnn_train.shape[2]), return_sequences=False),
        Dropout(dropout),
        Dense(units // 2, activation=activation_fn),
        Dropout(dropout),
        Dense(len(encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy')
    return model

In [50]:
def build_lstm_model_tuned(units, dropout, learning_rate, activation, optimizer):
    activation_fn = LeakyReLU() if activation == 'leaky_relu' else activation
    opt = {'adam': Adam, 'rmsprop': RMSprop, 'sgd': SGD}[optimizer](learning_rate=learning_rate)

    model = Sequential([
        LSTM(units, activation=activation_fn, input_shape=(1, X_rnn_train.shape[2]), return_sequences=False),
        Dropout(dropout),
        Dense(units // 2, activation=activation_fn),
        Dropout(dropout),
        Dense(len(encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy')
    return model

In [51]:
from tensorflow.keras.layers import Activation

In [52]:
def build_attbirnn_model_tuned(units, dropout, learning_rate, activation, optimizer):
    # Map activation functions
    activation_fn = LeakyReLU(alpha=0.3) if activation == 'leaky_relu' else activation
    opt = {'adam': Adam, 'rmsprop': RMSprop, 'sgd': SGD}[optimizer](learning_rate=learning_rate)

    input_layer = Input(shape=(1, X_rnn_train.shape[2]))
    birnn_layer = Bidirectional(LSTM(units, return_sequences=True, activation='tanh'))(input_layer)  # Use default LSTM activation (e.g., tanh)
    attention_layer = Attention()(birnn_layer)
    dense_layer = Dense(units // 2)(attention_layer)
    if activation == 'leaky_relu':
        dense_layer = LeakyReLU(alpha=0.3)(dense_layer)  # Apply LeakyReLU separately
    else:
        dense_layer = Activation(activation)(dense_layer)  # Apply other activations
    dense_layer = Dropout(dropout)(dense_layer)
    output_layer = Dense(len(encoder.classes_), activation='softmax')(dense_layer)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy')
    return model

In [53]:
def random_search_tuning(build_fn, param_distributions, X_train, y_train, X_val, y_val, n_iter=10, epochs=10, batch_size=32):
    best_model = None
    best_score = -np.inf
    best_params = None

    for params in ParameterSampler(param_distributions, n_iter=n_iter, random_state=42):
        print(f"Testing parameters: {params}")

        # Build model with current parameters
        model = build_fn(**params)

        # Prepare callbacks
        f1_callback = MacroF1Score(validation_data=(X_val, y_val))
        early_stopping = EarlyStopping(monitor='val_macro_f1', patience=10, mode='max', restore_best_weights=True)

        # Train the model
        model.fit(
            X_train, y_train,
            validation_split=0.1,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[f1_callback, early_stopping],
            verbose=1
        )

        # Get the best F1 score for this parameter set
        if f1_callback.best_macro_f1 > best_score:
            best_score = f1_callback.best_macro_f1
            best_model = model
            best_params = params

    print(f"Best F1 Score: {best_score:.4f}")
    print(f"Best Parameters: {best_params}")
    return best_model, best_score, best_params



In [54]:
param_distributions_ffn = {
    'neurons': randint(64, 256),  # FFN uses 'neurons'
    'dropout': uniform(0.2, 0.5),
    'learning_rate': uniform(0.0001, 0.01),
    'activation': ['relu', 'tanh', 'sigmoid', 'leaky_relu'],  # Activation functions
    'optimizer': ['adam', 'rmsprop', 'sgd']  # Optimizers
}
param_distributions_rnn = {
    'units': randint(64, 256),  # RNN uses 'units'
    'dropout': uniform(0.2, 0.5),
    'learning_rate': uniform(0.0001, 0.01),
    'activation': ['relu', 'tanh', 'sigmoid', 'leaky_relu'],
    'optimizer': ['adam', 'rmsprop', 'sgd']
}
param_distributions_lstm = {
    'units': randint(64, 256),  # LSTM uses 'units'
    'dropout': uniform(0.2, 0.5),
    'learning_rate': uniform(0.0001, 0.01),
    'activation': ['relu', 'tanh', 'sigmoid', 'leaky_relu'],
    'optimizer': ['adam', 'rmsprop', 'sgd']
}
param_distributions_attbirnn = {
    'units': randint(64, 256),  # AttBiRNN uses 'units'
    'dropout': uniform(0.2, 0.5),
    'learning_rate': uniform(0.0001, 0.01),
    'activation': ['relu', 'tanh', 'sigmoid', 'leaky_relu'],
    'optimizer': ['adam', 'rmsprop', 'sgd']
}

In [55]:
best_ffn_model, best_ffn_score, best_ffn_params = random_search_tuning(
    build_ffn_model_tuned,
    param_distributions_ffn,
    X_ffn_train, y_train, X_ffn_test, y_test,
    n_iter=10
)

Testing parameters: {'activation': 'sigmoid', 'dropout': 0.5982714934301164, 'learning_rate': 0.001934347898661638, 'neurons': 135, 'optimizer': 'adam'}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   loss
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.7589 - val_loss: 0.5699 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step loss: 
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6048 - val_loss: 0.5368 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step loss
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5850 - val_loss: 0.5283 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step loss
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5472 - val_loss: 0.5252 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step loss:
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5517 - val_loss: 0.5253 - val_macro_f1: 0.1824
Epoch 6/10
151/

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   lo
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.0736 - val_loss: 0.6370 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - loss
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6599 - val_loss: 0.6098 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 669us/step - loss: 0.
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6472 - val_loss: 0.5971 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - loss: 0.
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6238 - val_loss: 0.5904 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step loss: 0.6
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6135 - val_loss: 0.5850 - val_macro_f1: 0.1824
Epoch 6/10
1

...

151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step loss: 
Epoch 1 - Macro F1 Score: 0.2211
New Best Macro F1 Score: 0.2211
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.7839 - val_loss: 0.5208 - val_macro_f1: 0.2211
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step loss:
Epoch 2 - Macro F1 Score: 0.1936
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5600 - val_loss: 0.5157 - val_macro_f1: 0.1936
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step lo
Epoch 3 - Macro F1 Score: 0.2400
New Best Macro F1 Score: 0.2400
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5659 - val_loss: 0.5042 - val_macro_f1: 0.2400
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step loss
Epoch 4 - Macro F1 Score: 0.2256
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5490 - val_loss: 0.5094 - val_macro_f1: 0.2256
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step lo
Epoch 5 - Macro F1 Score: 0.2415
New Best Macro F1 Score: 0.2415
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5489 - val_lo

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.0569 - val_loss: 0.6379 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step loss: 
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7022 - val_loss: 0.6345 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step loss
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6880 - val_loss: 0.6318 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - loss
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6575 - val_loss: 0.6306 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - loss
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6765 - val_loss: 0.6285 - val_macro_f1: 0.1824
Epoch 6/10
151/151 ━━━━━━━━━━━

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step loss
Epoch 1 - Macro F1 Score: 0.1866
New Best Macro F1 Score: 0.1866
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.7979 - val_loss: 0.5499 - val_macro_f1: 0.1866
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step loss
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6060 - val_loss: 0.5639 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step loss:
Epoch 3 - Macro F1 Score: 0.1925
New Best Macro F1 Score: 0.1925
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6027 - val_loss: 0.5379 - val_macro_f1: 0.1925
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5971 - val_loss: 0.5500 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - loss: 0.
Epoch 5 - Macro F1 Score: 0.1833
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5962 - val_loss: 0.5387 - val_m

In [56]:
best_rnn_model, best_rnn_score, best_rnn_params = random_search_tuning(
    build_rnn_model_tuned,
    param_distributions_rnn,
    X_rnn_train, y_train, X_rnn_test, y_test,
    n_iter=10
)

Testing parameters: {'activation': 'sigmoid', 'dropout': 0.5982714934301164, 'learning_rate': 0.001934347898661638, 'optimizer': 'adam', 'units': 84}
Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   lo
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.8118 - val_loss: 0.5914 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step lo
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6337 - val_loss: 0.5486 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step loss
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5852 - val_loss: 0.5395 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step loss: 
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5660 - val_loss: 0.5274 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   lo
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5687 - val_loss: 0.5257 - val_macro_f1: 0.1824
Epoch 6/10
151/151 ━━━━━━━━━━━━━━

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   lo
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.0267 - val_loss: 0.6702 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step loss
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7103 - val_loss: 0.6497 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step loss: 0
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6974 - val_loss: 0.6415 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step loss
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6758 - val_loss: 0.6369 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step loss
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6974 - val_loss: 0.6345 - val_macro_f1: 0.1824
Epoch 6/10
151/151 ━━━━━━━━━

...

151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.6188 - val_loss: 0.5396 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step loss: 0
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5618 - val_loss: 0.5327 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step loss
Epoch 3 - Macro F1 Score: 0.1841
New Best Macro F1 Score: 0.1841
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5302 - val_loss: 0.5249 - val_macro_f1: 0.1841
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step loss
Epoch 4 - Macro F1 Score: 0.1841
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5493 - val_loss: 0.5291 - val_macro_f1: 0.1841
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step loss
Epoch 5 - Macro F1 Score: 0.1841
New Best Macro F1 Score: 0.1841
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5399 - val_loss

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   loss:
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.6402 - val_loss: 0.5479 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step loss
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5416 - val_loss: 0.5148 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 867us/step loss
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5297 - val_loss: 0.5202 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step loss:
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5331 - val_loss: 0.5128 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step loss
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5273 - val_loss: 0.5105 - val_macro_f1: 0.1824
Epoch 6/10
151/151 ━━━━━━━━

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   los
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.7028 - val_loss: 0.5638 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step loss
Epoch 2 - Macro F1 Score: 0.1850
New Best Macro F1 Score: 0.1850
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5821 - val_loss: 0.5480 - val_macro_f1: 0.1850
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step loss: 0
Epoch 3 - Macro F1 Score: 0.1859
New Best Macro F1 Score: 0.1859
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5674 - val_loss: 0.5423 - val_macro_f1: 0.1859
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step loss: 
Epoch 4 - Macro F1 Score: 0.1857
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5592 - val_loss: 0.5378 - val_macro_f1: 0.1857
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 979us/step loss:
Epoch 5 - Macro F1 Score: 0.1903
New Best Macro F1 Score: 0.1903
317/317 ━━━━━━━━━━━━━━━━━━

In [57]:
best_lstm_model, best_lstm_score, best_lstm_params = random_search_tuning(
    build_lstm_model_tuned,
    param_distributions_lstm,
    X_rnn_train, y_train, X_rnn_test, y_test,
    n_iter=10
)

Testing parameters: {'activation': 'sigmoid', 'dropout': 0.5982714934301164, 'learning_rate': 0.001934347898661638, 'optimizer': 'adam', 'units': 84}
Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   lo
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.8377 - val_loss: 0.5891 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step loss
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6285 - val_loss: 0.5521 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step loss
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6029 - val_loss: 0.5371 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5802 - val_loss: 0.5287 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step loss: 0
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5699 - val_loss: 0.5295 - val_macro_f1: 0.1824
Epoch 6/10
151/151 ━━━━━━━━━━━━━

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   los
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.2848 - val_loss: 0.6414 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6839 - val_loss: 0.6264 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6749 - val_loss: 0.6266 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step loss: 0
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6624 - val_loss: 0.6264 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step loss: 0
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6554 - val_loss: 0.6261 - val_macro_f1: 0.1824
Epoch 6/10
151/151 ━━━

...

151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step   los
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.6390 - val_loss: 0.5409 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5747 - val_loss: 0.5471 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5549 - val_loss: 0.5421 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - los
Epoch 4 - Macro F1 Score: 0.1862
New Best Macro F1 Score: 0.1862
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5315 - val_loss: 0.5235 - val_macro_f1: 0.1862
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - los
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5355 - val_loss: 0.5320 - val_macro_f1: 0.1824

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step  
Epoch 1 - Macro F1 Score: 0.1989
New Best Macro F1 Score: 0.1989
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.6364 - val_loss: 0.5327 - val_macro_f1: 0.1989
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - lo
Epoch 2 - Macro F1 Score: 0.1864
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5434 - val_loss: 0.5116 - val_macro_f1: 0.1864
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5284 - val_loss: 0.5167 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step loss: 
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5354 - val_loss: 0.5355 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5341 - val_loss: 0.5122 - val_macro_f1: 0.1824
Epoch 6/10
151/151 ━━━━━━━━━━━━

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step   l
Epoch 1 - Macro F1 Score: 0.1824
New Best Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.9732 - val_loss: 0.6235 - val_macro_f1: 0.1824
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6083 - val_loss: 0.6050 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6025 - val_loss: 0.5969 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - lo
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5801 - val_loss: 0.5908 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoch 5 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5953 - val_loss: 0.5865 - val_macro_f1: 0.1824
Epoch 6/10
151/151 ━━━━━━━━━━━━━

In [58]:
best_attbirnn_model, best_attbirnn_score, best_attbirnn_params = random_search_tuning(
    build_attbirnn_model_tuned,
    param_distributions_attbirnn,
    X_rnn_train, y_train, X_rnn_test, y_test,
    n_iter=10
)

Testing parameters: {'activation': 'sigmoid', 'dropout': 0.5982714934301164, 'learning_rate': 0.001934347898661638, 'optimizer': 'adam', 'units': 84}
Epoch 1/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step   los
Epoch 1 - Macro F1 Score: 0.1833
New Best Macro F1 Score: 0.1833
317/317 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.7662 - val_loss: 0.5238 - val_macro_f1: 0.1833
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss
Epoch 2 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5772 - val_loss: 0.5324 - val_macro_f1: 0.1824
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoch 3 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5326 - val_loss: 0.5091 - val_macro_f1: 0.1824
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:
Epoch 4 - Macro F1 Score: 0.1824
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5485 - val_loss: 0.5129 - val_macro_f1: 0.1824
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - los

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step   
Epoch 1 - Macro F1 Score: 0.1909
New Best Macro F1 Score: 0.1909
317/317 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.6143 - val_loss: 0.5244 - val_macro_f1: 0.1909
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - l
Epoch 2 - Macro F1 Score: 0.1890
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5278 - val_loss: 0.5154 - val_macro_f1: 0.1890
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step -
Epoch 3 - Macro F1 Score: 0.2357
New Best Macro F1 Score: 0.2357
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5174 - val_loss: 0.5133 - val_macro_f1: 0.2357
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step -
Epoch 4 - Macro F1 Score: 0.1849
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5252 - val_loss: 0.5344 - val_macro_f1: 0.1849
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - los
Epoch 5 - Macro F1 Score: 0.1840
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.5272 - val_loss: 0.5101 - val_macro_f1: 0.1840
Epoch 6/10
151

In [59]:
from sklearn.metrics import classification_report
import numpy as np

def evaluate_model(model, X_test, y_test, model_name):
    # Predict on the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Generate classification report
    print(f"\n{model_name} Classification Report:")
    print(classification_report(y_test, y_pred_classes, target_names=encoder.classes_))

In [60]:
evaluate_model(best_ffn_model, X_ffn_test, y_test, "FFN")

151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

FFN Classification Report:
               precision    recall  f1-score   support

     negative       0.18      0.09      0.12        69
      neutral       0.25      0.01      0.01       143
     positive       0.36      0.08      0.14       485
very negative       0.21      0.15      0.18        84
very positive       0.86      0.98      0.91      4048

     accuracy                           0.83      4829
    macro avg       0.37      0.26      0.27      4829
 weighted avg       0.77      0.83      0.78      4829

